In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import torch
import yaml
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [5]:
def seed_everything(seed=40):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
seed_everything()

In [ ]:
DATA_YAML_PATH = "data.yaml"

with open(DATA_YAML_PATH, "r", encoding="utf-8") as f:
    data_yaml = yaml.safe_load(f)

DATASET_PATH = os.path.dirname(DATA_YAML_PATH)
TRAIN_IMAGES = os.path.join(DATASET_PATH, data_yaml["train"].replace("../", ""))
VALID_IMAGES = os.path.join(DATASET_PATH, data_yaml["val"].replace("../", ""))

print(DATASET_PATH)
print(TRAIN_IMAGES)
print(VALID_IMAGES)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
class ObstacleDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = glob(os.path.join(image_dir, "*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        return self.transform(img) if self.transform else img


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

train_loader = DataLoader(ObstacleDataset(TRAIN_IMAGES, transform), batch_size=16, shuffle=True)
valid_loader = DataLoader(ObstacleDataset(VALID_IMAGES, transform), batch_size=16, shuffle=False)


def show_sample_images(image_loader):
    sample_images = next(iter(image_loader))

    fig, ax = plt.subplots(2, 3, figsize=(12, 8))

    for i, img in enumerate(sample_images[:6]):
        ax[i // 3, i % 3].imshow(img.permute(1, 2, 0).numpy())
        ax[i // 3, i % 3].axis("off")

    plt.show()


print("훈련 데이터 샘플")
show_sample_images(train_loader)

In [ ]:
IMAGE_DIR = "/kaggle/input/pothole-detection-challenge/train/images"
IMAGE_PATHS = sorted(glob(os.path.join(IMAGE_DIR, "*.jpg")))

IMG_PATH = random.choice(IMAGE_PATHS)
LABEL_PATH = IMG_PATH.replace("images", "labels").replace(".jpg", ".txt")

img = cv2.imread(IMG_PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w, _ = img.shape

if os.path.exists(LABEL_PATH):
    with open(LABEL_PATH, "r") as f:
        for line in f.readlines():
            cls, cx, cy, bw, bh = map(float, line.strip().split())
            x1 = int((cx - bw / 2) * w)
            y1 = int((cy - bh / 2) * h)
            x2 = int((cx + bw / 2) * w)
            y2 = int((cy + bh / 2) * h)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 5)
            cv2.putText(img, f"Class {int(cls)}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)
else:
    print("라벨 파일이 존재하지 않습니다:", LABEL_PATH)

# 시각화
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis("off")
plt.title(os.path.basename(IMG_PATH))
plt.show()

In [ ]:
model = YOLO("/kaggle/working/pothole_yolov8_train/weights/best.pt")

val_results = model.val(data=DATA_YAML_PATH, split="val")

print("검증 데이터 평가 결과:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")

In [2]:
def train_yolov8m(data_path='data.yaml', epochs=100, img_size=640, batch_size=16, project='runs/train', name='exp'):
    """
    Trains a YOLOv8m model.

    Args:
        data_path (str): Path to the YAML file containing dataset configuration.
        epochs (int): Number of training epochs.
        img_size (int): Image size for training.
        batch_size (int): Batch size for training.
        project (str): Project name for saving results.
        name (str): Experiment name for saving results within the project.
    """
    # Load a YOLOv8m model
    # You can load a pre-trained model for fine-tuning
    model = YOLO('yolov8m.pt')  # yolov8m.pt is the pre-trained medium model

    # If you want to train from scratch (less common for yolov8m, but possible)
    # model = YOLO('yolov8m.yaml') # Create a new model from scratch configuration

    print(f"Starting YOLOv8m training with the following parameters:")
    print(f"  Data: {data_path}")
    print(f"  Epochs: {epochs}")
    print(f"  Image Size: {img_size}")
    print(f"  Batch Size: {batch_size}")
    print(f"  Project: {project}")
    print(f"  Name: {name}")

    # Train the model
    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        project=project,
        name=name,
        # You can add more arguments here based on your needs, e.g.:
        # device='0',         # GPU device (e.g., '0' for first GPU, or 'cpu')
        # patience=50,        # Early stopping patience
        # resume=False,       # Resume training from last checkpoint
        # lr0=0.01,           # Initial learning rate
        # lrf=0.001,          # Final learning rate (lr0 * lrf)
        # optimizer='auto',   # Optimizer: 'SGD', 'Adam', 'AdamW', 'RMSProp', 'auto'
        # workers=8,          # Number of DataLoader workers
        # val=True,           # Validate during training
        # cache=False,        # Cache images for faster training
        # cos_lr=False,       # Cosine learning rate scheduler
        # exist_ok=False,     # Don't overwrite existing results directory
        # single_cls=False,   # Treat all classes as a single class
        # save_period=10,     # Save checkpoint every 'save_period' epochs
    )